<a href="https://colab.research.google.com/github/DiegoLuis62/Ciencias-de-datos---PF/blob/main/Heart_Disease_Cleanse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Carga y exploración inicial del dataset**

In [41]:
# Bibliotecas estándar
import os
import sys

# Manipulación de datos
import pandas as pd

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    precision_recall_curve,
    average_precision_score
)
from sklearn.impute import SimpleImputer
import lightgbm as lgb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
dataset_path = "/content/drive/My Drive/10 semestre/PF-Dataset"
df = pd.read_csv(dataset_path + "/heart_2022_with_nans.csv")
# Link del dataset : https://www.kaggle.com/datasets/kamilpytlak/personal-key-indicators-of-heart-disease

In [ ]:
print(df.shape)
print("")
df.head()


In [ ]:
df.info()

# **2. Análisis de valores nulos: cantidad, porcentaje y tipo de datos**

En este análisis, se identifican las columnas que contienen valores nulos dentro del dataset. Se calcula la cantidad y el porcentaje de datos faltantes en cada variable, además de determinar su tipo de dato (numérico o categórico). Esto permite evaluar el impacto de los valores ausentes y definir estrategias adecuadas para la limpieza y el tratamiento de los datos.

In [ ]:
missing_values = df.isnull().sum()

# Filtrar solo las columnas con NaN
missing_columns = missing_values[missing_values > 0]

# Obtener los tipos de datos de esas columnas
dtypes = df.dtypes[missing_columns.index]

# Crear un DataFrame con la información
missing_info = pd.DataFrame({
    "Missing Values": missing_columns,
    "Percentage": (missing_columns / len(df)) * 100,
    "Data Type": dtypes
})


missing_info

In [ ]:
for col in df: # Verificar si hay una columna o atributo que no aporte, ejemplo si tiene 1 subnivel, no nos sirve
  print(f"Columna {col}: {df[col].nunique() } subniveles ")

In [ ]:
# Verificar cantidad de filas duplicadas en el dataset original
duplicados = df.duplicated().sum()

print(f"Número de filas duplicadas en el dataset original: {duplicados}")
df.drop_duplicates(inplace=True)



Acabo de eliminar 157 datos duplicados del dataset original para evitar tener que eliminarlos uno por uno al copiar los demás datasets.

#**3) Limpieza de los Datasets**



#Limpieza de Dataset por Eliminación



In [ ]:
df_cleaned = df.dropna()  # Elimina filas con valores nulos
print(f"Filas originales: {df.shape[0]}")
print(f"Filas después de limpieza: {df_cleaned.shape[0]}")



#Limpieza de Dataset Usando Moda, Mediana

In [ ]:



df_imputado = df.copy()

# Identificar variables categóricas y numéricas
categorical_vars = df_imputado.select_dtypes(include=['object']).columns
binary_vars = [var for var in categorical_vars if df_imputado[var].nunique() == 2]  # Variables binarias
num_vars = df_imputado.select_dtypes(include=['float64']).columns

# Imputación con moda para categóricas y binarias
imputer_moda = SimpleImputer(strategy="most_frequent")
df_imputado[categorical_vars] = imputer_moda.fit_transform(df_imputado[categorical_vars])

# Imputación con mediana para variables numéricas
imputer_mediana = SimpleImputer(strategy="median")
df_imputado[num_vars] = imputer_mediana.fit_transform(df_imputado[num_vars])

# Guardar dataset imputado
df_imputado.to_csv("dataset_imputado.csv", index=False)




In [ ]:
print(f"Filas originales: {df.shape[0]}")
print(f"Filas después de imputar: {df_imputado.shape[0]}")

# **4) Comparación de columnas numéricas del dataset original, eliminación de datos nulos e imputación.**

**Dataset con datos Originales**

In [ ]:

df.describe()


**Dataset con datos nulos eliminados**

In [ ]:
df_cleaned.describe()

**Dataset con datos Imputados**

In [ ]:

df_imputado.describe()

# **Análisis Comparativo de Métodos para el Manejo de Datos Faltantes**

En el presente análisis, se compararon dos estrategias para tratar los datos faltantes en el dataset: la eliminación de registros con valores nulos y la imputación mediante la moda (para datos categóricos) y la mediana (para datos numéricos). A continuación, se presentan los hallazgos clave.

# 1️⃣ Eliminación de Datos Faltantes

Se redujo significativamente la cantidad de datos, pasando de ~434,205 registros a 246,022 en algunas columnas, representando una pérdida aproximada del 43.3% de la información.

La media de variables clave, como el Índice de Masa Corporal (BMI), pasó de 28.53 en el dataset original a 28.67 tras la eliminación de datos, lo que sugiere que los valores eliminados tenían una distribución diferente al conjunto de datos restante.

Se observó una disminución en la dispersión de los datos (medida a través de la desviación estándar), lo que indica una posible pérdida de variabilidad en la muestra.

# 2️⃣ Imputación con Moda y Mediana

Se conservaron 445,132 registros, manteniendo la totalidad de la muestra sin pérdida de información.

La media del BMI fue 28.41, mucho más cercana al valor original (28.53) en comparación con la eliminación de datos.

La desviación estándar disminuyó ligeramente en comparación con el dataset original, pero sin alterar significativamente la variabilidad inherente a los datos.

# 📌 Conclusión

El análisis comparativo evidencia que la eliminación de datos genera una pérdida considerable de información y puede introducir sesgos en la muestra al alterar las estadísticas descriptivas de las variables. En contraste, la imputación con moda y mediana preserva la estructura del dataset y mantiene los valores estadísticos más cercanos a los originales.

Por lo tanto, para evitar la pérdida de información valiosa y minimizar el sesgo en futuros análisis, se recomienda la imputación de valores faltantes en lugar de la eliminación de registros.

# **5) Análisis de datos**

In [ ]:


# Suponiendo que tienes tu DataFrame llamado 'df'
numerical_columns = [
    "PhysicalHealthDays", "MentalHealthDays", "SleepHours",
    "HeightInMeters", "WeightInKilograms", "BMI"
]

# Crear boxplots para cada columna numérica
for col in numerical_columns:
    plt.figure(figsize=(6, 4))  # Tamaño del gráfico
    sns.boxplot(x=df[col])
    plt.title(f'{col} - Datos atípicos', fontsize=14)
    plt.xlabel(col)
    plt.savefig(f"{col}_boxplot.png")  # Guarda la imagen
    plt.show()







In [ ]:


# Convertir variables categóricas a numéricas usando codificación ordinal
df_encoded = df_imputado.copy()

categorical_columns = [
    "State", "Sex", "GeneralHealth", "LastCheckupTime", "PhysicalActivities",
    "RemovedTeeth", "HadHeartAttack", "HadAngina", "HadStroke", "HadAsthma",
    "HadSkinCancer", "HadCOPD", "HadDepressiveDisorder", "HadKidneyDisease",
    "HadArthritis", "HadDiabetes", "DeafOrHardOfHearing", "BlindOrVisionDifficulty",
    "DifficultyConcentrating", "DifficultyWalking", "DifficultyDressingBathing",
    "DifficultyErrands", "SmokerStatus", "ECigaretteUsage", "ChestScan",
    "RaceEthnicityCategory", "AgeCategory", "AlcoholDrinkers", "HIVTesting",
    "FluVaxLast12", "PneumoVaxEver", "TetanusLast10Tdap", "HighRiskLastYear", "CovidPos"
]

for col in categorical_columns:
    df_encoded[col] = df_encoded[col].astype("category").cat.codes  # Asigna códigos numéricos a las categorías

# Calcular la matriz de correlación
corr_matrix = df_encoded.corr()

# Crear el heatmap con valores numéricos visibles y mayor tamaño
plt.figure(figsize=(20, 15))
sns.heatmap(
    corr_matrix,
    cmap="coolwarm",
    annot=True,
    fmt=".2f",
    linewidths=0.5,
    annot_kws={"size": 8}  # Reduce el tamaño de los números
)

# Configurar etiquetas
plt.xticks(rotation=90)  # Rotar etiquetas del eje X para mejor visualización
plt.yticks(rotation=0)   # Mantener las etiquetas del eje Y horizontales
plt.title("Mapa de calor de correlaciones entre variables", fontsize=20)

# Mostrar el heatmap
plt.show()



In [ ]:
# Crear el gráfico de conteo con df_imputado
ax = sns.countplot(data=df_imputado, x='HadHeartAttack', hue='Sex')

# Función para personalizar el gráfico
def customize_plot(ax, title, xlabel, ylabel, width, height):
    ax.set_title(title, fontsize=16)
    ax.set_xlabel(xlabel, fontsize=14)
    ax.set_ylabel(ylabel, fontsize=14)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.gcf().set_size_inches(width, height)  # Ajustar tamaño de la figura

# Aplicar la personalización
customize_plot(ax, "Genders and Heart Attack", "Had Heart Attack", "Individuals", 12, 10)

# Guardar el gráfico en un archivo
scatter_fig = ax.get_figure()
scatter_fig.savefig('genderHeartAttack.png', dpi=300, bbox_inches='tight')

# Mostrar el gráfico
plt.show()

In [ ]:
# Asegúrate de que 'AgeCategory' sea una columna categórica
df_imputado['AgeCategory'] = df_imputado['AgeCategory'].astype('category')

# Crear el gráfico de barras
ax = sns.countplot(data=df_imputado, x='AgeCategory', hue='HadHeartAttack')

# Rotar etiquetas del eje X para mejorar legibilidad
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")

# Personalizar el gráfico
customize_plot(ax, "Heart Attacks and Age Groups", "Had Heart Attack", "Individuals", 12, 10)

# Añadir etiquetas a las barras
for c in ax.containers:
    ax.bar_label(c)

# Guardar el gráfico
scatter_fig = ax.get_figure()
scatter_fig.savefig('countplotByAge.png')

# Mostrar el gráfico
plt.show()

# **6) Árbol RandomForestClassifier**


In [ ]:
df_imputado.info()

In [ ]:
for col in df: # Verificar si hay una columna o atributo que no aporte, ejemplo si tiene 1 subnivel, no nos sirve
  print(f"Columna {col}: {df_imputado[col].nunique() } subniveles ")

In [ ]:

# 1. Conteo de clases
class_counts = df_imputado['HadHeartAttack'].value_counts()

# 2. Gráfico de barras
plt.figure(figsize=(10, 5))
sns.barplot(x=class_counts.index, y=class_counts.values, palette="viridis")
plt.title("Distribución de Ataques Cardíacos (Conteo)")
plt.xlabel("HadHeartAttack")
plt.ylabel("Número de casos")
plt.show()

# 3. Gráfico de pastel (porcentajes)
plt.figure(figsize=(6, 6))
plt.pie(class_counts, labels=class_counts.index, autopct='%1.1f%%', colors=['#ff9999','#66b3ff'])
plt.title("Proporción de Ataques Cardíacos")
plt.show()

In [ ]:
# Obtener valores únicos de todas las columnas en df_imputado
for col in df_imputado.columns:
    print(f"\n Valores únicos en la columna {col}: {df_imputado[col].unique()}\n")


In [ ]:


# 1. Preparación de Datos
# --------------------------------------------------
target_column = "HadHeartAttack"

# Convertir la variable objetivo a categórica directamente
df_imputado[target_column] = df_imputado[target_column].astype('category')

# Definir variables predictoras (X) y la variable objetivo (y)
X = df_imputado.drop(columns=[target_column])
y = df_imputado[target_column]  # Mantenemos como categoría

# Identificar columnas categóricas en X
categorical_columns = X.select_dtypes(include=['object']).columns.tolist()
X[categorical_columns] = X[categorical_columns].astype('category')

# 2. División de Datos
# --------------------------------------------------
# Dividir manteniendo las categorías originales
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=87, stratify=y
)

# 3. Configuración y Entrenamiento del Modelo
# --------------------------------------------------
lgb_model = lgb.LGBMClassifier(
    boosting_type='goss',
    n_estimators=1000,
    learning_rate=0.01,
    max_depth=-1,
    num_leaves=63,
    min_data_in_leaf=30,
    feature_fraction=0.8,
    lambda_l1=0.1,
    lambda_l2=0.1,
    # class_weight='balanced',
    class_weight={'No': 1, 'Yes': 5},
    random_state=42,
    objective='binary',  # Asegurar que es para clasificación binaria
    metric='binary_logloss'
)

# Entrenamiento con early stopping (una sola vez)
lgb_model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric='binary_logloss',
    categorical_feature=categorical_columns,
    callbacks=[lgb.early_stopping(stopping_rounds=50)]
)

# 4. Predicción y Evaluación
# --------------------------------------------------
# Realizar predicciones (ya están en las categorías originales)
y_pred = lgb_model.predict(X_test)

# Calcular métricas directamente con las categorías
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='Yes')
recall = recall_score(y_test, y_pred, pos_label='Yes')
f1 = f1_score(y_test, y_pred, pos_label='Yes')

# Imprimir métricas
print("\nMétricas de Evaluación:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# 5. Visualización
# --------------------------------------------------
# Matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues",
            xticklabels=['No Heart Attack', 'Heart Attack'],
            yticklabels=['No Heart Attack', 'Heart Attack'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Matriz de Confusión')
plt.show()

# Curva Precision-Recall (Opcional)

y_probs = lgb_model.predict_proba(X_test)[:, 1]
precision_curve, recall_curve, _ = precision_recall_curve(y_test, y_probs, pos_label='Yes')
ap_score = average_precision_score(y_test, y_probs, pos_label='Yes')

plt.figure(figsize=(10, 6))
plt.plot(recall_curve, precision_curve, label=f'AP Score: {ap_score:.2f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Curva Precision-Recall')
plt.legend()
plt.show()

# **Mal Rendimiento**
Como el rendimiento no es tan bueno, ajustaremos el dataset conservando solo las columnas relevantes, ya que las demás están afectando negativamente el rendimiento

In [ ]:
columnas_a_eliminar = [
    # variable objectivo. así que no debe estar en x para predecirla así misma
    'HadHeartAttack',
    'State',  # Ubicación geográfica no es un factor médico directo
    'DeafOrHardOfHearing',  # Problemas auditivos no relacionados
    'BlindOrVisionDifficulty',  # Problemas visuales no relacionados
    'HIVTesting',  # No directamente relacionado con salud cardíaca
    'RemovedTeeth',  # Salud dental no es predictor cardíaco
    'ChestScan',  # Es un examen, no un factor de riesgo
    'FluVaxLast12',  # Vacuna de gripe no es relevante
    'PneumoVaxEver',  # Vacuna neumococo no es relevante
    'CovidPos',  # Muy reciente para tener datos concluyentes
    'HeightInMeters',  # Mejor usar BMI que combina altura/peso
    'WeightInKilograms',  # Mejor usar BMI
    'DifficultyConcentrating',  # Síntoma muy genérico
    'DifficultyDressingBathing',  # Movilidad no específica cardíaca
    'DifficultyErrands'  # Movilidad no específica cardíaca
]


variables_clave = [


    # Factores demográficos básicos
    'Sex',
    'AgeCategory',

    # Salud general
    'GeneralHealth',
    'PhysicalHealthDays',
    'MentalHealthDays',
    'BMI',

    # Factores de riesgo cardiovascular
    'HadAngina',
    'HadStroke',
    'HadAsthma',
    'HadCOPD',
    'HadDiabetes',
    'HadKidneyDisease',
    'HadArthritis',

    # Hábitos de vida
    'SmokerStatus',
    'ECigaretteUsage',
    'AlcoholDrinkers',
    'PhysicalActivities',
    'SleepHours',

    # Comorbilidades relevantes
    'HadDepressiveDisorder',
    'HadSkinCancer',  # Algunos estudios muestran correlación

    # Exámenes médicos
    'LastCheckupTime',
    'HighRiskLastYear',

    # Dificultades físicas relacionadas
    'DifficultyWalking'  # Puede indicar problemas circulatorios
]

In [ ]:
y_test.unique()

In [ ]:
y_train.unique()

In [ ]:
X.shape

In [ ]:
X.info()

In [ ]:


# 1. Preparación de Datos
# --------------------------------------------------
target_column = "HadHeartAttack"

# Convertir la variable objetivo a categórica directamente
df_imputado[target_column] = df_imputado[target_column].astype('category')

# Definir variables predictoras (X) y la variable objetivo (y)
X = df_imputado.drop(columns=columnas_a_eliminar)  # Elimina todo de una vez
y = df_imputado[target_column]  # Mantenemos como categoría

# Identificar columnas categóricas en X
categorical_columns = X.select_dtypes(include=['object']).columns.tolist()
X[categorical_columns] = X[categorical_columns].astype('category')

# 2. División de Datos
# --------------------------------------------------
# Dividir manteniendo las categorías originales
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=87, stratify=y
)

# 3. Configuración y Entrenamiento del Modelo
# --------------------------------------------------
lgb_model = lgb.LGBMClassifier(
    boosting_type='goss',  # Gradient-based One-Side Sampling
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=15,
    num_leaves=31,
    min_child_samples=50,
    reg_alpha=0.5,
    reg_lambda=0.5,
    class_weight={'No': 1, 'Yes': 5},
    random_state=42,
    objective='binary',
    metric='aucpr',
    n_jobs=-1,
    importance_type='gain',
    min_data_in_leaf=100,
    cat_smooth=20,
    extra_trees=True
)

# Entrenamiento con early stopping (una sola vez)
lgb_model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric='binary_logloss',
    categorical_feature=categorical_columns,
    callbacks=[lgb.early_stopping(stopping_rounds=10)]
)

# 4. Predicción y Evaluación
# --------------------------------------------------
# Realizar predicciones (ya están en las categorías originales)
y_pred = lgb_model.predict(X_test)

# Calcular métricas directamente con las categorías
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='Yes')
recall = recall_score(y_test, y_pred, pos_label='Yes')
f1 = f1_score(y_test, y_pred, pos_label='Yes')

# Imprimir métricas
print("\nMétricas de Evaluación:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# 5. Visualización
# --------------------------------------------------
# Matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues",
            xticklabels=['No Heart Attack', 'Heart Attack'],
            yticklabels=['No Heart Attack', 'Heart Attack'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Matriz de Confusión')
plt.show()

# Curva Precision-Recall (Opcional)

y_probs = lgb_model.predict_proba(X_test)[:, 1]
precision_curve, recall_curve, _ = precision_recall_curve(y_test, y_probs, pos_label='Yes')
ap_score = average_precision_score(y_test, y_probs, pos_label='Yes')

plt.figure(figsize=(10, 6))
plt.plot(recall_curve, precision_curve, label=f'AP Score: {ap_score:.2f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Curva Precision-Recall')
plt.legend()
plt.show()

In [ ]:
X.info()

In [ ]:
X[["AgeCategory"]]

# **Randont Forest Pero con las Filas nulas eliminadas "HadHearAttack = No" e imputados en HadHearAttack =SI"**

In [ ]:


# 1. Crear copia del dataframe original
df= df.copy()

# 2. Separar los casos positivos (Yes) y negativos (No)
positivos = df[df['HadHeartAttack'] == 'Yes']
negativos = df[df['HadHeartAttack'] == 'No']

# 3. Eliminar filas con valores nulos SOLO en los negativos (No)
negativos_limpios = negativos.dropna()

# 4. Combinar los positivos completos con los negativos limpios
df_Prueba = pd.concat([positivos, negativos_limpios], axis=0)

# 5. Imputación simple para las variables numéricas y categóricas restantes
# Seleccionar columnas numéricas y categóricas
numeric_cols = df_Prueba.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = df_Prueba.select_dtypes(include=['object', 'category']).columns

# Imputar numéricos con la mediana (menos sensible a outliers)
imputer_num = SimpleImputer(strategy='median')
df_Prueba[numeric_cols] = imputer_num.fit_transform(df_Prueba[numeric_cols])

# Imputar categóricas con la moda (valor más frecuente)
imputer_cat = SimpleImputer(strategy='most_frequent')
df_Prueba[categorical_cols] = imputer_cat.fit_transform(df_Prueba[categorical_cols])

# Verificación final
print(f"Registros originales: {len(df_Prueba)}")
print(f"Registros después del procesamiento: {len(df_Prueba)}")
print(f"Distribución de clases:\n{df_Prueba['HadHeartAttack'].value_counts(normalize=True)}")

In [ ]:


# 1. Conteo de clases
class_counts = df_Prueba['HadHeartAttack'].value_counts()

# 2. Gráfico de barras
plt.figure(figsize=(10, 5))
sns.barplot(x=class_counts.index, y=class_counts.values, palette="viridis")
plt.title("Distribución de Ataques Cardíacos (Conteo)")
plt.xlabel("HadHeartAttack")
plt.ylabel("Número de casos")
plt.show()

# 3. Gráfico de pastel (porcentajes)
plt.figure(figsize=(6, 6))
plt.pie(class_counts, labels=class_counts.index, autopct='%1.1f%%', colors=['#ff9999','#66b3ff'])
plt.title("Proporción de Ataques Cardíacos")
plt.show()

In [ ]:


# 1. Preparación de Datos
# --------------------------------------------------
target_column = "HadHeartAttack"

# Convertir la variable objetivo a categórica directamente
df_Prueba[target_column] = df_Prueba[target_column].astype('category')

# Definir variables predictoras (X) y la variable objetivo (y)
X = df_Prueba.drop(columns=columnas_a_eliminar)  # Elimina todo de una vez
y = df_Prueba[target_column]  # Mantenemos como categoría

# Identificar columnas categóricas en X
categorical_columns = X.select_dtypes(include=['object']).columns.tolist()
X[categorical_columns] = X[categorical_columns].astype('category')

# 2. División de Datos
# --------------------------------------------------
# Dividir manteniendo las categorías originales
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=87, stratify=y
)

# 3. Configuración y Entrenamiento del Modelo
# --------------------------------------------------
lgb_model = lgb.LGBMClassifier(
    boosting_type='goss',  # Gradient-based One-Side Sampling
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=15,
    num_leaves=31,
    min_child_samples=50,
    reg_alpha=0.5,
    reg_lambda=0.5,
    class_weight={'No': 1, 'Yes': 5},
    random_state=42,
    objective='binary',
    metric='aucpr',
    n_jobs=-1,
    importance_type='gain',
    min_data_in_leaf=100,
    cat_smooth=20,
    extra_trees=True
)

# Entrenamiento con early stopping (una sola vez)
lgb_model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric='binary_logloss',
    categorical_feature=categorical_columns,
    callbacks=[lgb.early_stopping(stopping_rounds=10)]
)

# 4. Predicción y Evaluación
# --------------------------------------------------
# Realizar predicciones (ya están en las categorías originales)
y_pred = lgb_model.predict(X_test)

# Calcular métricas directamente con las categorías
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='Yes')
recall = recall_score(y_test, y_pred, pos_label='Yes')
f1 = f1_score(y_test, y_pred, pos_label='Yes')

# Imprimir métricas
print("\nMétricas de Evaluación:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# 5. Visualización
# --------------------------------------------------
# Matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues",
            xticklabels=['No Heart Attack', 'Heart Attack'],
            yticklabels=['No Heart Attack', 'Heart Attack'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Matriz de Confusión')
plt.show()

# Curva Precision-Recall (Opcional)

y_probs = lgb_model.predict_proba(X_test)[:, 1]
precision_curve, recall_curve, _ = precision_recall_curve(y_test, y_probs, pos_label='Yes')
ap_score = average_precision_score(y_test, y_probs, pos_label='Yes')

plt.figure(figsize=(10, 6))
plt.plot(recall_curve, precision_curve, label=f'AP Score: {ap_score:.2f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Curva Precision-Recall')
plt.legend()
plt.show()